In [44]:
import pandas
from os import listdir

def getFiles(s):
    return map(lambda x: f"{s}/{x}", listdir(s))
# filenames = [*getFiles("out-results"), *getFiles("out-shuffle"), *getFiles("out-structure-10-10")]
filenames = getFiles("out")
files = [pandas.read_csv(file, sep=';')
         for file in filenames if '/coq-' in file]

df = pandas.concat(files)

rest_filenames = getFiles("out-rerun")
rest_files = [pandas.read_csv(file, sep=';')
         for file in rest_filenames if '/coq-' in file]
df2 = pandas.concat(rest_files)
df2 = df2.loc[~df2['Algorithm'].isin(['StructuredUnification', 'StructuredSplitUnification', 'SplitTypeUnification', 'SplitTypeSplitUnification', 'SimpleUnification', 'SimpleSplitUnification'])]

In [65]:
combined = pandas.concat([df, df2])
trimmed = combined[['File Name', 'Algorithm', 'Ranking Factor', 'Size Factor', 'Index', 'Line', 'Character', 'Keywords Before']]
trimmed.loc[trimmed['Ranking Factor'].isnull(), 'Ranking Factor'] = 0
trimmed.loc[trimmed['Size Factor'].isnull(), 'Size Factor'] = 0
groups = trimmed.groupby(['Algorithm', 'Ranking Factor', 'Size Factor'])

In [67]:
from collections import defaultdict as dd
allresults: dict[tuple, dict[int, tuple[int, dict[int, int]]]] = {}  # groupname -> lemmaPosition -> (count, (resultIndex -> count))
data = []
for groupName, frame in groups:
    if (not (
        "Structure" in groupName[0]
        and groupName[1] == 2
        and groupName[2] == 5)): continue
    line = -1
    ch = -1
    lemmaPosition = -1
    rewritedata = dd(lambda: dd(int))
    apply_data = dd(lambda: dd(int))
    rewrite_positionCounts = dd(int)
    apply_positionCounts = dd(int)
    allresults[groupName] = {}
    for resultIndex, l, c, kw in frame[['Index', 'Line', 'Character', 'Keywords Before']].values:
        if (type(kw) != str): continue
        if line != l:
            line = l
            ch = c
            lemmaPosition = 0
        if ch != c:
            if ch + 1 == c:
                lemmaPosition += 1
            else:
                lemmaPosition = 0
            ch = c
        if "rewrite" in kw:
            rewritedata[lemmaPosition][resultIndex] += 1
            rewrite_positionCounts[lemmaPosition] += 1
        else:
            apply_data[lemmaPosition][resultIndex] += 1
            apply_positionCounts[lemmaPosition] += 1
        # print(resultIndex, l, c, lemmaPosition)
    
    rewrite_grades = dd(int)
    apply_grades = dd(int)
    for thisdata, positionCounts, grades, kw in [
        # (rewritedata, rewrite_positionCounts, rewrite_grades, 'rewrite'),
        (apply_data, apply_positionCounts, apply_grades, 'apply')
        ]:
        for lemmaPosition, results in thisdata.items():
            attempts = positionCounts[lemmaPosition]
            allresults[groupName].update({lemmaPosition: (attempts, results)})
            grades[lemmaPosition] = sum(map(lambda j: results[j] / (j + 1), range(10))) / attempts
        a = sum(map(lambda x: grades[x] / (x + 1), range(len(grades))))
        b = sum(map(lambda x: 1 / (x+1), range(len(grades))))
        if b == 0:
            print(groupName[:2], a, "failed")
            continue
        score = 100 * ( a / b )
        data.append((*groupName[:1], kw, round(score, 2)))
result_df = pandas.DataFrame(data, columns=['Algorithm', 'Keyword', 'Score'])
# result_df.sort_values(by=['Score'], inplace=True, ascending=False)
result_df

,Algorithm,Keyword,Score
0,StructuredSplitUnification,apply,48.21
1,StructuredTypeEvaluation,apply,52.97
2,StructuredUnification,apply,52.97


In [22]:
tableLength = 8
for k, v in allresults.items():
    lmao = []
    for lemmaPosition, (amount, results) in v.items():
        if lemmaPosition > tableLength:
            break
        first = results[0]
        second = results[1]
        third = results[2]
        rest = sum(map(lambda x: results[x], range(3, 10)))
        fail = amount - sum([first, second, third, rest])
        passed = amount - fail
        lmao.append([*map(lambda x: round(x * 100 / amount, 2), [first, second, third, rest, passed, fail])])
    pnt = { 'sep': ' & ', 'end': '\\\\\n' }
    print(r'\begin{table}')
    print(r'\small')
    print('\\begin{tabular}{|r|', *map(lambda x: "r", range(tableLength + 1)), r'|}\hline', sep="")
    # print(f'\multicolumn{{{tableLength + 2}}}{{l}}{{\\normalsize', k, '}', '\\\\', '\\hline')
    print("Prefix", *range(tableLength + 1), sep=' & ', end='\\\\\\hline\n')
    for x, n in enumerate(["1st","2nd","3rd","4-10th","1-10th","Fail"]):
        print(n, *map(lambda l: f'{l[x]:.2f}', lmao), sep=pnt['sep'], end=r"\\")
        if x == 3:
            print(r"\hline", end="")
        print()
    print('\\hline\\end{tabular}')
    print(r'\vspace{0.4em}')
    print(r'\caption{Results from', k[0], "" if int(k[1]) == 0 else f' with ranking factor {k[1]} and size factor {k[2]}', '}')
    print(r'\end{table}')
    print()

\begin{table}
\small
\begin{tabular}{|r|rrrrrrrrr|}\hline
Prefix & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8\\\hline
1st & 0.67 & 8.90 & 26.39 & 34.08 & 35.80 & 37.88 & 42.11 & 49.81 & 54.06\\
2nd & 0.55 & 6.57 & 11.77 & 12.80 & 13.39 & 14.89 & 16.08 & 16.02 & 14.95\\
3rd & 0.31 & 4.61 & 6.17 & 6.54 & 6.45 & 6.34 & 6.63 & 5.16 & 4.41\\
4-10th & 5.04 & 22.77 & 22.53 & 16.62 & 16.07 & 13.69 & 10.56 & 8.07 & 6.93\\\hline
1-10th & 6.57 & 42.84 & 66.85 & 70.04 & 71.71 & 72.80 & 75.37 & 79.07 & 80.36\\
Fail & 93.43 & 57.16 & 33.15 & 29.96 & 28.29 & 27.20 & 24.63 & 20.93 & 19.64\\
\hline\end{tabular}
\vspace{0.4em}
\caption{Results from Basic  }
\end{table}

\begin{table}
\small
\begin{tabular}{|r|rrrrrrrrr|}\hline
Prefix & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8\\\hline
1st & 0.02 & 7.07 & 25.47 & 33.17 & 35.75 & 38.24 & 45.32 & 52.83 & 57.01\\
2nd & 0.01 & 3.06 & 9.71 & 9.78 & 10.57 & 11.73 & 13.62 & 14.07 & 12.57\\
3rd & 0.05 & 2.96 & 6.62 & 6.24 & 6.83 & 6.82 & 5.59 & 4.47 & 4.39\\
4-10th & 0.23 & 14.6

In [7]:
def getFiles(s):
    return map(lambda x: f"{s}/{x}", listdir(s))
filenames = getFiles("out")
time_files = [pandas.read_csv(file, sep=';')
         for file in filenames if 'scores' in file]
time_df = pandas.concat(time_files)
time_df = time_df[['Algorithm', 'Ranking Factor', 'Size Factor', 'Time']].groupby(['Algorithm', 'Ranking Factor', 'Size Factor']).mean()
time_df

Time
Algorithm                  Ranking Factor Size Factor              
SimpleSplitUnification     0              0            37066.867725
SimpleUnification          0              0            37234.719577
SplitTypeSplitUnification  0              0            38778.756614
SplitTypeUnification       0              0            38336.867725
StructuredSplitUnification 1              1            36884.560847
                                          2            36769.100529
                                          5            36546.698413
                           2              1            36413.544974
                                          2            36414.238095
                                          5            36447.825397
                           5              1            36577.201058
                                          2            37026.126984
                                          5            36805.740741
StructuredUnification      1              1            41746.952381
                                          2            41251.878307
                                          5            41276.947090
                           2              1            40761.862434
                                          2            40524.735450
                                          5            39850.402116
                           5              1            39296.005291
                                          2            38557.693122
                                          5            37764.100529